<a href="https://colab.research.google.com/github/anish-sk/CS6910_Assignment1/blob/master/src/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade wandb
!wandb login 6746f968d95eb71e281d6c7772a0469574430408

     |████████████████████████████████| 2.0MB 11.9MB/s 
     |████████████████████████████████| 133kB 39.1MB/s 
     |████████████████████████████████| 102kB 10.8MB/s 
     |████████████████████████████████| 163kB 37.9MB/s 
     |████████████████████████████████| 71kB 9.7MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=a3d08f8d63dd7aa6955dbe40ac530cdd5c29618ec0b14744f87a30eb15140b04
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=9e81b8456b35c27f1364502392aa1b84c00724ed6f190c78013da204f967e677
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
Successfully built subprocess32 pathtools
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [2]:
# Init wandb
import wandb

wandb.init(project="assignment1", entity="abisheks")

import numpy as np
from matplotlib import pyplot as plt
# Loading the fashion mnist dataset
from keras.datasets import fashion_mnist

# Setting seed value
np.random.seed(1)

# Load dataset (train data and test data)
(trainX, trainy), (testX, testy) = fashion_mnist.load_data()

# Summarize loaded dataset
print('Train: X=%s, y=%s' % (trainX.shape, trainy.shape))
print('Test: X=%s, y=%s' % (testX.shape, testy.shape))

wandb: Currently logged in as: abisheks (use `wandb login --relogin` to force relogin)


4423680/4422102 [==============================] - 0s 0us/step
Train: X=(60000, 28, 28), y=(60000,)
Test: X=(10000, 28, 28), y=(10000,)


In [3]:
# Number of classes in the Fashion-MNIST dataset
N_CLASSES = np.unique(trainy).shape[0]    # 10 as known from the keras documentation

# Captions/Labels for the output classes present in Fashion-MNIST dataset
IMG_LABELS = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']


def getSampleImages(nClass, imgLabels, X, y, nSamples):
  '''
  The function takes few samples of each class from the dataset provided and passes it to the WANDB for it log the images

  Arguments :
    nClass -- Number of output classes in the dataset
    imgLabels -- List of labels for the output classes (numbered from 0 to nClass - 1)
    X -- The input data containing images in the form of matrices
    y -- The output data containing the class to which an input belongs
    nSamples -- Number of samples of each class to be taken. If that many samples not present in dataset, maximum number of samples present (from that class) will be taken

  Returns :
    -- None --
  '''

  # Initialise empty list to store the input data sampled from each class
  sampleImgsX = [[] for _ in range(nClass)]

  # Take 3 sample images from each class
  for i in range(y.shape[0]):
    if len(sampleImgsX[y[i]]) < nSamples :
      sampleImgsX[y[i]].append(X[i])


  # Getting a list of sample images of each class to be saved to wandb
  sampleImgsList = []
  for i in range(nClass):
    for j in range(3):
      sampleImgsList.append(wandb.Image(sampleImgsX[i][j], caption = imgLabels[i]))

  np.random.shuffle(sampleImgsList)
  wandb.log({"example" : sampleImgsList})


# Question 1 : Show 3 sample images from training set of downloaded Fashion-MNIST dataset in WANDB
getSampleImages(N_CLASSES, IMG_LABELS, trainX, trainy, 3)

In [4]:
def sigmoid(x):
  #Calculates the sigmoid function
  return np.exp(-np.logaddexp(0, -x))

def softmax(x):
  #Calculates the softmax function
  e_x = np.exp(x - np.max(x))
  return e_x / e_x.sum()

def linear(W, X, b):
  #Calculates the linear function
  return W @ X + b

In [11]:
def initialize_network(n_L, preActFns_L, actFns_L):
  '''
  The function initializes the neural network and the appropriate parameters
  
  Arguments :
  n_L -- an array whose ith element represents the number of neurons in the ith layer (0 - Input Layer, last element - Output Layer)
  preActFns_L -- an array who ith element is the Pre Activation function of (i+1)th layer of the neural network
  actFns_L -- an array who ith element is the Activation function of (i+1)th layer of the neural network

  Returns :
  network -- the initialized network as an array of dictionaries for the hidden and output layers of the neural network
  '''

  L = len(n_L)-1

  assert(L >= 1)
  assert(len(preActFns_L) == L)
  assert(len(actFns_L) == L)

  network = list()
  for i in range(1,L+1):
    # Dictionary for each layer representing it's constituents
    layer = {'weights':np.random.randn(n_L[i], n_L[i-1])*0.01,    # Weight matrix for (i-1)th to ith layer transition
             'biases':np.zeros((n_L[i],1)),                       # Bias vector for (i-1)th to ith layer transition
             'pre_activation_fn':preActFns_L[i-1],                # Pre-activation function for neurons of ith layer
             'activation_fn':actFns_L[i-1],                       # Activation function for neurons of ith layer
             'no_neurons':n_L[i],                                 # Number of neurons in ith layer
             'cache': []                                          # Array of cached pre-activation and activation output for each layer to be used in back-propogation (will be filled in forward-propogation)
            }
    network.append(layer)	
  return network


wandb.config.update({"n_hidden_layers": 1, "size_hidden_layer":4})   # Setting the hyperparameters in the wandb
L = wandb.config['n_hidden_layers']+1    # Number of hidden layerws + Output layer in the neural network
n_L = [wandb.config['size_hidden_layer']] * (L+1)    # List of number of neurons in the neural network

n_L[0] = trainX.shape[1] * trainX.shape[2]
n_L[L] = N_CLASSES

pre_act_fns_L = [linear] * L    # List of Pre-activation functions of the hidden layers and output layer
act_fns_L = [sigmoid] * (L-1) + [softmax]   # List of Activation functions of the hidden layers and output layer

network = initialize_network(n_L, pre_act_fns_L, act_fns_L)
print(network)

[{'weights': array([[ 0.0032712 , -0.00161081,  0.0048104 , ..., -0.00826681,
         0.00903995, -0.00647184],
       [ 0.00390153, -0.00705483, -0.00440369, ..., -0.00819731,
        -0.00948209,  0.00021856],
       [-0.00088553,  0.00687713,  0.00025081, ...,  0.01351832,
         0.00033313, -0.00273922],
       [ 0.00202317, -0.0158685 ,  0.0043207 , ...,  0.01137318,
        -0.00314253,  0.00430464]]), 'biases': array([[0.],
       [0.],
       [0.],
       [0.]]), 'pre_activation_fn': <function linear at 0x7fcab0d5a050>, 'activation_fn': <function sigmoid at 0x7fcab0cd6560>, 'no_neurons': 4, 'cache': []}, {'weights': array([[-0.0095319 , -0.01199536,  0.00450282, -0.00555743],
       [-0.01330772, -0.00169307, -0.00654462, -0.00255923],
       [ 0.00540258, -0.00703787, -0.00949956, -0.0021139 ],
       [-0.01660067,  0.00929078,  0.00799257, -0.00748297],
       [-0.00802371,  0.00667073, -0.00229169, -0.00059968],
       [ 0.01247042, -0.00737659,  0.00351498,  0.01337076],

In [12]:
def pre_activation(H_prev, W, b, pre_activation_fn):
  # Calculates the pre-activation output and caches the required values. Returns the output and cache.
  print(W.shape, H_prev.shape, b.shape)
  A = pre_activation_fn(W, H_prev, b)
  

  assert(A.shape == (W.shape[0], H_prev.shape[1]))
  pre_act_cache = A   # Caching the pre-activation ouptut to be used in backpropogation

  return A, pre_act_cache

def feedforward_neuron(H_prev, W, b, activation_fn, pre_activation_fn):
  # Calculates the activation output (using the pre-activation function above) and caches the required values. Returns the output and cache.

  A, pre_activation_cache = pre_activation(H_prev, W, b, pre_activation_fn)
  H = activation_fn(A)

  assert (H.shape == (W.shape[0], H_prev.shape[1]))
  cache = (pre_activation_cache, H)   # Caching the pre-activation and activation output to use it in back-propogation

  return H, cache

In [14]:
def forward_propogation(X, network):
    """
    Implement forward propagation for the given neural network
    
    Arguments :
    X -- Input data as a matrix where ith column is the ith input data from the training example
    network -- Given neural network (as an array of dictionaries)
    
    Returns :
    H -- Output from the neural network
    """

    H = X                         # Initialising H to input
    L = len(network)              # Number of (hidden + output) layers in the neural network
    
    for l in range(0, L):
        H_prev = H 
        H, cache = feedforward_neuron(H_prev, network[l]['weights'], network[l]['biases'], network[l]['activation_fn'], network[l]['pre_activation_fn'])
        network[l]['cache'] = cache
    
    assert(H.shape == (network[L-1]['no_neurons'], X.shape[1]))
            
    return H

trainX_reshaped = trainX.reshape((trainX.shape[1]*trainX.shape[2], trainX.shape[0]))
HL = forward_propogation(trainX_reshaped, network)          # HL -- output from the neural network
print(HL)

(4, 784) (784, 60000) (4, 1)
(10, 4) (4, 60000) (10, 1)
[[1.63623888e-06 1.64214243e-06 1.65513263e-06 ... 1.66853923e-06
  1.64997392e-06 1.64199663e-06]
 [1.64557863e-06 1.63653689e-06 1.64898198e-06 ... 1.66792420e-06
  1.66762769e-06 1.66397609e-06]
 [1.66756311e-06 1.65414991e-06 1.68007935e-06 ... 1.67213409e-06
  1.65896730e-06 1.65587192e-06]
 ...
 [1.71943325e-06 1.71678149e-06 1.68770052e-06 ... 1.67342856e-06
  1.70357590e-06 1.70685706e-06]
 [1.69490682e-06 1.70102191e-06 1.68429280e-06 ... 1.67400855e-06
  1.68519566e-06 1.70744315e-06]
 [1.64849654e-06 1.63985281e-06 1.65425632e-06 ... 1.66805580e-06
  1.66327672e-06 1.64883753e-06]]
